# Problema del commesso viaggatore Simmetrico (TSP)

## Definizione
### Dato un grafo $G(V,A)$ a cui è associato un costo $d_{ij}$ ad ogni arco $(i,j) \in A$. Il problema del commesso viaggiatore consiste nel cercare il circuito Hamiltoniano di costo minimo.
### - Un ciclo Hamiltoniano è un ciclo che attraversa tutti i nodi del grafo una ed una sola volta.
### - Il costo di un ciclo Hamiltoniano è dato dalla somma dei costi degli archi che lo compongono.

## Definizione

## Variabili di decisione

### <span style="color:purple">$x_{ij} \quad (i,j) \in A $ </span>  - variabile binaria uguale a $1$ se l'arco $(i,j)$ appartiene al circuito hamiltoniano, 0 altrimenti.


### Funzione obiettivo
Minimizza il costo totale del circuito hamiltoniano

\begin{equation}
\text{Min} \quad Z = \sum_{(i,j) \in A} d_{ij} \cdot x_{ij}
\tag{0}
\end{equation}

### Constraints 
- **Vincoli di assegnamento**. In una soluzione ammissibile (circuito hamiltoniano) ogni nodo deve avere esattamente un arco entrante ed esattamente un arco uscente.

\begin{equation}
\sum_{i \in V, \ i \neq j} x_{ij} = 1 \quad \quad j \in V 
\tag{1}
\end{equation}

\begin{equation}
\sum_{i \in V, \ i \neq j} x_{ji} = 1 \quad \quad j \in V 
\tag{2}
\end{equation}

- **Vincoli di assenza di sottogiri**. In una soluzione ammissibile (circuito hamiltoniano) non ci possono essere cicli su un sottoinsime proprio dell'insieme dei nodi $V$.

\begin{equation}
\sum_{i,j \in S, \ (i \neq j)}x_{i,j} \leq |S|-1 \quad \quad  S \subset V
\tag{3}
\end{equation}

In [1]:
def read_coords(path):
    coords = []
    with open(path, "r") as f:
        for i in range(0, 3):
                line = f.readline()
        line = f.readline()
        line = line.strip('\n')
        data = line.split(':')
        dimension = int(data[1])
        for i in range(0, 4):
            line = f.readline()
        for i in range(0, dimension):
            line = f.readline()
            data = line.split()
            coord = [float(data[1]), float(data[2])]
            coords.append(coord)
    return coords, dimension

In [2]:
coords, dimension = read_coords("burma14.txt")

In [3]:
 def dist(node_0, node_1):
        """
        Euclidean distance between two nodes.
        """
        coord_0, coord_1 = coords[node_0], coords[node_1]
        return math.sqrt((coord_0[0] - coord_1[0]) ** 2 + (coord_0[1] - coord_1[1]) ** 2)

In [4]:
import gurobipy as gp
from gurobipy import GRB

import numpy as np

# Inizializza il modelo
mod = gp.Model('TSP')

# Crea le varibili di decisione
vars = mod.addVars(dist.keys(), obj=dist, vtype=GRB.BINARY, name='x')

ModuleNotFoundError: No module named 'gurobipy'

In [ ]:
nodes = [i for i in range(len(coords))]
outstar = mod.addConstrs(vars.sum(i, '*') == 1 for i in nodes)

instar = mod.addConstrs(vars.sum('*', i) == 1 for i in nodes)

In [ ]:
def subtourelim(model, where):
    if where == GRB.Callback.MIPSOL:
        # preleva la soluzione corrente
        vals = model.cbGetSolution(model._vars)
        selected = gp.tuplelist((i,j) for i, j in model._vars.keys() if vals[i,j] > 0.5)
        # cerca il ciclo di lunghezza minima nella soluzione
        tour = subtour(selected)
        if len(tour) < len(capoluoghi):
            # aggiunge il vincolo di eliminazione di sottogiro
            model.cbLazy(gp.quicksum(model._vars[i,j] for i in tour for j in tour if i != j )
                         <= len(tour)-1)

def subtour(edges):
    unvisited = nodes[:]
    cycle = nodes[:] 
    while unvisited:  # true if list is non-empty
        thiscycle = []
        neighbors = unvisited
        while neighbors:
            current = neighbors[0]
            thiscycle.append(current)
            unvisited.remove(current)
            neighbors = [j for i, j in edges.select(current, '*')
                         if j in unvisited]
        if len(thiscycle) <= len(cycle):
            cycle = thiscycle # New shortest subtour
    return cycle

In [ ]:
mod._vars = vars
mod.Params.lazyConstraints = 1
mod.optimize(subtourelim)

In [ ]:
vals = mod.getAttr('x', vars)
selected = gp.tuplelist((i, j) for i, j in vals.keys() if vals[i, j] > 0.5)

tour = subtour(selected)
assert len(tour) == len(capoluoghi)

In [ ]:
tour

In [ ]:
import matplotlib.pyplot as plt
import networkx as nx

Graph = nx.DiGraph()
list_nodes = list(range(1, num_nodes+1))
Graph.add_nodes_from(list_nodes)
for i,j in links:
    Graph.add_edge(i,j)

# Definisce la posizione di ogni nodo
node_pos = coords

# Crea la lista di archi della soluzione
red_edges = [(i,j) for i,j in links if x[i,j].x > 0]

# Crea la lista di nodi della soluzione
shortestpath = [ i for i,j in links if x[i,j].x > 0 ]
shortestpath.append(destination)

# Colora di rosso i nodi della soluzione e di verde gli altri
node_col = ['green' if not node in shortestpath else 'red' for node in Graph.nodes()]
# Colora di rosso gli archi della soluzione e di nero gli altri
edge_col = ['black' if not edge in red_edges else 'red' for edge in Graph.edges()]

# Draw the nodes
nx.draw_networkx(Graph,node_pos, node_color= node_col, node_size=450)

# Draw the edges
nx.draw_networkx_edges(Graph, node_pos,edge_color= edge_col)

# Draw the edge labels
nx.draw_networkx_edge_labels(Graph, node_pos, edge_labels=cost)

# Show the plot
plt.show()